<a href="https://colab.research.google.com/github/blackgreenc/java/blob/master/KerasModel_Java_pj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)
# MNIST Datasets 불러오기 
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

2.7.0
11501568/11490434 [==============================] - 0s 0us/step


In [2]:
# input data normalization
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
# model create
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
                                        
])

# model compile
mlp_model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [4]:
# model
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [5]:
# model training
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2600 - accuracy: 0.9272
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1124 - accuracy: 0.9670
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0780 - accuracy: 0.9764
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0588 - accuracy: 0.9821
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0461 - accuracy: 0.9855


In [6]:
# model evaluation
mlp_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0765 - accuracy: 0.9755 - 501ms/epoch - 2ms/step


[0.07648034393787384, 0.9754999876022339]

In [7]:
# keras model convert to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model) # create converter 
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpzq7g6ncz/assets


In [8]:
# save
with open('./keras_model.tflite', 'wb') as f:
    f.write(tflite_model) 

In [9]:
tf.saved_model.save(mlp_model, "/mlp_model/")

INFO:tensorflow:Assets written to: /mlp_model/assets


INFO:tensorflow:Assets written to: /mlp_model/assets


In [12]:
# ve the Keras model in HDF5 format.
mlp_model.save("./mlp_model.h5")
mlp_model.save("./mlp_model/")

INFO:tensorflow:Assets written to: ./mlp_model/assets


INFO:tensorflow:Assets written to: ./mlp_model/assets


In [13]:
saved_model = tf.keras.models.load_model("./mlp_model/")

In [14]:
h5_model = tf.keras.models.load_model("./mlp_model.h5")

In [17]:
# keras model convert to tflite model
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/") # create converter 
tflite_model = converter.convert()